The traffic accident data is the key to this analysis. To make sure it's current, instead of downloading a static .csv file it, along with several other datasets from the City of Nashville's OpenData portal, will be periodically downloaded and updated via APIs.

To avoid too many pull requests, all of the data will be pulled here and the relevant information will be exported for use in a separate notebook for EDA.

** This notebook will be revisited and edited as data needs may change for this analysis.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Needed to pip install sodapy first
from sodapy import Socrata 

**Data Source:** https://data.nashville.gov/Police/Traffic-Accidents/6v6w-hpcw/about_data

In [ ]:
client = Socrata("data.nashville.gov", None)

In [ ]:
crashes_export = client.get("6v6w-hpcw", limit=200000)
crashes_raw = pd.DataFrame.from_records(crashes_export)
crashes_raw.head()

In [ ]:
crashes_raw.info()

Now that the DataFrame has been created, some columns can be removed as they're not relevant to this analysis.

In [ ]:
crashes_raw = crashes_raw.drop(['reporting_officer', 'collision_type', 'illuaccidemination', 'harmfulcodes', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', 'weather'], axis=1)

In [ ]:
crashes_raw['date_and_time'] = pd.to_datetime(crashes_raw['date_and_time'])
crashes_raw = crashes_raw.astype({'number_of_motor_vehicles': 'float',
                    'number_of_injuries': 'float',
                    'number_of_fatalities': 'float',
                    'hit_and_run': 'bool',
                    'property_damage': 'bool'})

In [ ]:
crashes_raw.info()

For some initial cleaning, all text fields should be converted to the same case, in this case UPPER

In [ ]:
crashes_raw['collision_type_description'] = crashes_raw['collision_type_description'].astype(str).str.upper()
crashes_raw['illumination_description'] = crashes_raw['illumination_description'].astype(str).str.upper()
crashes_raw['harmfuldescriptions'] = crashes_raw['harmfuldescriptions'].astype(str).str.upper()
crashes_raw['street_address'] = crashes_raw['street_address'].astype(str).str.upper()
crashes_raw['city'] = crashes_raw['city'].astype(str).str.upper()
crashes_raw['state'] = crashes_raw['state'].astype(str).str.upper()
crashes_raw['precinct'] = crashes_raw['precinct'].astype(str).str.upper()
crashes_raw['weather_description'] = crashes_raw['weather_description'].astype(str).str.upper()
crashes_raw['property_damage'] = crashes_raw['property_damage'].astype(str).str.upper()

In [ ]:
crashes_raw.head()

Last, this analysis is only concerned with crashes in the vicinity of Main Street / Gallatin Avenue / Gallatin Pike in East Nashville & Madison. To filter the data, a list of points needs to be created to essentially trace the path of the corridor. From there, only data points within, say, a 150' radius of that line will be included for analysis. I can't find a better way to do this than selecting points on Google Maps and manually entering the coordinates into a list. Refer to `mapping.ipynb`

In [ ]:
east_nash_crashes = crashes_raw[crashes_raw['zip'].isin(['37206', '37216', '37115'])]
east_nash_crashes

The resulting table is exported to a .csv file for use in the EDA notebook.

In [ ]:
east_nash_crashes.to_csv('../data/clean/east_nash_crashes.csv')

Now repeat the process for active ROW permits.<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Active-Right-of-Way-Permits/a5tp-4w2v/about_data

In [ ]:
row_permits_export = client.get("a5tp-4w2v", limit=2000000)
row_permits_raw = pd.DataFrame.from_records(row_permits_export)
row_permits_raw.head()

In [ ]:
row_permits_raw.info()

In [ ]:
row_permits_raw = row_permits_raw.drop(['status', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', ':@computed_region_b9k3_hpc2'], axis=1)

In [ ]:
row_permits_raw['initiated_date'] = pd.to_datetime(row_permits_raw['initiated_date'])
row_permits_raw['scheduled_start'] = pd.to_datetime(row_permits_raw['scheduled_start'])
row_permits_raw['scheduled_end'] = pd.to_datetime(row_permits_raw['scheduled_end'])

In [ ]:
row_permits = row_permits_raw[row_permits_raw['zip'].isin(['37206', '37216', '37115'])]
row_permits

In [ ]:
row_permits.to_csv('../data/clean/row_permits.csv')

...and Nashville 311 calls<br><br>
Source: https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data

In [ ]:
nash_311_export = client.get("7qhx-rexh", limit=2000000)
nash_311_raw = pd.DataFrame.from_records(nash_311_export)
nash_311_raw.head()

In [ ]:
nash_311_raw.info()

In [ ]:
nash_311_raw = nash_311_raw.drop(['oem_id', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', ':@computed_region_yf9r_ed6g', ':@computed_region_fvtq_wnma', ':@computed_region_s8bq_67w7', ':@computed_region_v67z_xm3t', ':@computed_region_kh5x_g7w5', ':@computed_region_cfa7_hbpz', ':@computed_region_sjpq_96s8', ':@computed_region_gisn_y5cm', ':@computed_region_b9k3_hpc2'], axis=1)

In [ ]:
nash_311_raw['date_time_opened'] = pd.to_datetime(nash_311_raw['date_time_opened'])
nash_311_raw['date_time_closed'] = pd.to_datetime(nash_311_raw['date_time_closed'])

In [ ]:
nash_311 = nash_311_raw[nash_311_raw['incident_zip_code'].isin(['37206', '37216', '37115'])]
nash_311

In [ ]:
nash_311.to_csv('../data/clean/nash_311.csv')

...and building permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7/about_data

In [ ]:
bdlg_permits_export = client.get("3h5w-q8b7", limit=2000000)
bdlg_permits_raw = pd.DataFrame.from_records(bdlg_permits_export)
bdlg_permits_raw.head()

In [ ]:
bdlg_permits_raw.info()

In [ ]:
bdlg_permits_raw = bdlg_permits_raw.drop([':@computed_region_f73m_vb2k', ':@computed_region_cfa7_hbpz', ':@computed_region_gisn_y5cm', ':@computed_region_v3ji_vzam', ':@computed_region_c9xn_skx3', ':@computed_region_sjpq_96s8', ':@computed_region_kh5x_g7w5', ':@computed_region_yf9r_ed6g', ':@computed_region_fvtq_wnma', ':@computed_region_p6sk_2acq', ':@computed_region_b9k3_hpc2', ':@computed_region_gxvr_9jxz'], axis=1)

In [ ]:
bdlg_permits_raw['date_entered'] = pd.to_datetime(bdlg_permits_raw['date_entered'])
bdlg_permits_raw['date_issued'] = pd.to_datetime(bdlg_permits_raw['date_issued'])

In [ ]:
bdlg_permits = bdlg_permits_raw[bdlg_permits_raw['zip'].isin(['37206', '37216', '37115'])]
bdlg_permits

In [ ]:
bdlg_permits.to_csv('../data/clean/bldg_permits.csv')

... and MNPD calls<br><br>
Source: https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Calls-for-Servic/kwnd-qrrm/about_data

In [ ]:
mnpd_export = client.get("kwnd-qrrm", limit=5000000)
mnpd_raw = pd.DataFrame.from_records(mnpd_export)
mnpd_raw.head()

...and Short Term Rental Permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm/about_data

In [ ]:
str_export = client.get("kwnd-qrrm", limit=100000)
strs = pd.DataFrame.from_records(str_export)
strs.head()